In [1]:
# !pip install SpeechRecognition -qq

In [2]:
import speech_recognition as sr
from jiwer import compute_measures
import string
import numpy as np
r = sr.Recognizer()

In [3]:
from sk import get_files,predict,process_text,load_model
audios = get_files("/content/test-bahasa/",[".wav"],recurse=True)
audios

(#780) [PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-woman-boleh berubah.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-news-audio-910.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ192-000005.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-streaming-kini didominasi ahli upm.wav'),PosixPath('/content/test-bahasa/wattpad-audio-wattpad-541.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-akuk.wav'),PosixPath('/content/test-bahasa/-home-husein-speech-bahasa-news-audio-2188.wav'),PosixPath('/content/test-bahasa/wattpad-audio-wattpad-0.wav'),PosixPath('/content/test-bahasa/iium-audio-iium-80.wav'),PosixPath('/content/test-bahasa/iium-audio-iium-5.wav')...]

In [4]:
no_digit_text = [i.with_suffix(".txt").read_text() for i in audios]
with_digit_text = [i.with_suffix(".txt").read_text() for i in get_files("/content/test-bahasa-norm/",[".wav"],recurse=True)]

In [5]:
def google(audio):
    with sr.AudioFile(str(audio)) as source:
        a = r.record(source)

    try:
        text = r.recognize_google(a, language = 'ms')
    except Exception as e:
        print(source,e)
        text = ""
    text = process_text(text)
    return text

In [6]:
%%time
from pqdm.threads import pqdm

g_texts = pqdm(audios,google,n_jobs=4)

SUBMITTING | : 100%|██████████| 780/780 [00:00<00:00, 45045.40it/s]
PROCESSING | : 100%|██████████| 780/780 [00:55<00:00, 14.16it/s]
COLLECTING | : 100%|██████████| 780/780 [00:00<00:00, 310807.25it/s]

CPU times: user 2.48 s, sys: 4.89 s, total: 7.37 s
Wall time: 55.1 s


In [7]:
g_d_wer = [compute_measures(i,j)["wer"] for i,j in zip(with_digit_text,g_texts)]
g_d_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(with_digit_text,g_texts)]
np.mean(g_d_wer),np.mean(g_d_cer) # with digit

(0.15330368944496492, 0.054922303921270864)

In [8]:
g_n_wer = [compute_measures(i,j)["wer"] for i,j in zip(no_digit_text,g_texts)]
g_n_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(no_digit_text,g_texts)]
np.mean(g_n_wer),np.mean(g_n_cer) # no digit

(0.1611180648863363, 0.06063404045488549)

In [9]:
# warm up
predict(audios[0],decoder="/content/out.trie.klm");

saved to: /root/.sk/models/conformer_small.onnx
loading model
loading language model
Total input path: 1
Total audio found(.wav): 1
start prediction


100%|██████████| 1/1 [00:00<00:00, 19.84it/s]


In [10]:
%%time
texts = predict(audios,decoder="/content/out.trie.klm")["texts"]

loading language model
Total input path: 780
Total audio found(.wav): 780
start prediction


100%|██████████| 780/780 [00:59<00:00, 13.16it/s]

CPU times: user 3min 51s, sys: 112 ms, total: 3min 51s
Wall time: 59.3 s


In [11]:
from words2num import normalize
n_texts = [normalize(i) for i in texts]
# with digit
sk_d_wer = [compute_measures(i,j)["wer"] for i,j in zip(with_digit_text,n_texts)]
sk_d_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(with_digit_text,n_texts)]
np.mean(sk_d_wer),np.mean(sk_d_cer) 

(0.13969373962274065, 0.03908335854120645)

In [12]:
# no digit
sk_n_wer = [compute_measures(i,j)["wer"] for i,j in zip(no_digit_text,texts)]
sk_n_cer = [compute_measures([ii for ii in i],[jj for jj in j])["wer"] for i,j in zip(no_digit_text,texts)]
np.mean(sk_n_wer),np.mean(sk_n_cer)

(0.13508409027804705, 0.036035160999825004)

In [13]:
import pandas as pd
pd.set_option('max_colwidth', 400)
df = pd.DataFrame(zip(*[audios,no_digit_text,texts,n_texts,with_digit_text,g_texts]))
df.columns = ["path","no_digit_label","our","our_with_digit","with_digit_label","google"]
df.head()

,path,no_digit_label,our,our_with_digit,with_digit_label,google
0,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-woman-boleh berubah.wav,sebut perkataan boleh berubah,sebut perkataan boleh berubah,sebut perkataan boleh berubah,sebut perkataan boleh berubah,sebut perkataan boleh berubah
1,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-910.wav,kerajaan negeri yang ditetapkan sultan johor untuk melantik menteri besar yang,kerajaan negeri yang ditetapkan sultan johor untuk melantik menteri besar yang,kerajaan negeri yang ditetapkan sultan johor untuk melantik menteri besar yang,kerajaan negeri yang ditetapkan sultan johor untuk melantik menteri besar yang,kerajaan negeri yang ditetapkan sultan johor untuk melantik menteri besar yang
2,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ192-000005.wav,apa yang salah dengan mengatakan daging babi bagus di islam,apa yang salah dengan mengatakan daging babi bagus di islam,apa yang salah dengan mengatakan daging babi bagus di islam,apa yang salah dengan mengatakan daging babi bagus di islam,apa yang salah dengan mengatakan daging babi bagus di islam
3,/content/test-bahasa/-home-husein-speech-bahasa-streaming-kini didominasi ahli upm.wav,kini didominasi ahli upm,kini didominasi ahli upm,kini didominasi ahli upm,kini didominasi ahli upm,kini didominasi ahli upm
4,/content/test-bahasa/wattpad-audio-wattpad-541.wav,ini mau ganti baju abangku sayang,ini mahu ganti baju abangku sayang,ini mahu ganti baju abangku sayang,ini mau ganti baju abangku sayang,ini mau ganti baju abangku sayang


In [14]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [15]:
df[df['with_digit_label'].apply(hasNumbers)].head(10)

,path,no_digit_label,our,our_with_digit,with_digit_label,google
6,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-2188.wav,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi,empat puluh lima ahli belia yang dilantik itu menjadi penyambung legasi,45 ahli belia yang dilantik itu menjadi penyambung legasi,45 ahli belia yang dilantik itu menjadi penyambung legasi,45 ahli belia yang dilantik itu menjadi penyambung legasi
18,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-897.wav,phuket kochi dari rm tiga ratus sembilan puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,buket koci dari rm tiga ratus sembila puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,buket koci dari rm 300 sembila 9 bali dari rm 629 nanjing dari rm 589,phuket kochi dari rm359 bali dari rm629 nanjing dari rm589,phuket kochi dari rm359 bali dari rm600 29 nanjing dari rm589
21,/content/test-bahasa/wattpad-audio-wattpad-96.wav,ketika mereka berada didepan ruang kelas dua belas ipa lima tatapan mereka langsung,ketika mereka berada di depan ruang kelas dua belas ip lima tetapan mereka langsung,ketika mereka berada di depan ruang kelas 12 ip 5 tetapan mereka langsung,ketika mereka berada didepan ruang kelas 12 ipa 5 tatapan mereka langsung,ketika mereka berada di depan ruang kelas 12 ipa 58 michael ang
28,/content/test-bahasa/wattpad-audio-wattpad-7.wav,bandung dua ribu tujuh belas,bandung dua ribu tujuh belas,bandung 2017,bandung 2017,bandung 2017
59,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ198-000005.wav,adakah ia baik untuk seorang wanita berumur dua belas tahun,adakah ia baik untuk seorang wanita berumur dua belas tahun,adakah ia baik untuk seorang wanita berumur 12 tahun,adakah ia baik untuk seorang wanita berumur 12 tahun,adakah ia baik untuk seorang wanita berumur 12 tahun
78,/content/test-bahasa/iium-audio-iium-4.wav,menyesal tulis tajuk guna nombor roman kena google nak tengok bentuk nombor lapan macam mana,menyesal tulis tajuk guna nombor roman kerangugae nak tengok bentuk nombor lapan macam mana,menyesal tulis tajuk guna nombor roman kerangugae nak tengok bentuk nombor 8 macam mana,menyesal tulis tajuk guna nombor roman kena google nak tengok bentuk nombor 8 macam mana,menyesal tulis tajuk guna nombor roman ok google nak tengok bentuk nombor 8 macam mana
92,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ259-000005.wav,bagaimana boleh lima belas juta penyamun hujat tidak menjadi bangsa atau agama,bagaimana boleh lima belas juta penyamun hujat tidak menjadi bangsa atau agama,bagaimana boleh 15000000 penyamun hujat tidak menjadi bangsa atau agama,bagaimana boleh 15 juta penyamun hujat tidak menjadi bangsa atau agama,bagaimana boleh 15 juta penyamun hujan tidak menjadi bangsa atau agama
97,/content/test-bahasa/wattpad-audio-wattpad-10.wav,ipa tiga arahnya tepat mengenai punggung salah satu siswi dikelas tersebut,ipa tiga arahnya tepat mengenai punggung salah satu siswi di kelas tersebut,ipa 3 arahnya tepat mengenai punggung salah 1 siswi di kelas tersebut,ipa 3 arahnya tepat mengenai punggung salah satu siswi dikelas tersebut,ip3 arahnya tepat mengenai punggung salah satu siswa di kelas tersebut
136,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-394.wav,semenjak ianya ditubuhkan lebih dua puluh tahun yang lalu,semenjak ianya ditubuhkan lebih dua puluh tahun yang lalu,semenjak ianya ditubuhkan lebih 20 tahun yang lalu,semenjak ianya ditubuhkan lebih 20 tahun yang lalu,semenjak ianya ditubuhkan lebih 20 tahun yang lalu
140,/content/test-bahasa/iium-audio-iium-61.wav,aku ni seorang perempuan dan baru sembilan belas tahun,aku ni seorang perempuan dan baru sembian belas tahun,aku ni seorang perempuan dan baru sembian 10 tahun,aku ni seorang perempuan dan baru 19 tahun,aku ni seorang perempuan dan baru 15 tahun


# No digit dataset comparison

In [16]:
df = pd.DataFrame(zip(*[audios,no_digit_text,texts,g_texts,sk_n_cer,g_n_cer]))
df.columns = ["path","no_digit_label","our","google","our_no_digit_cer","google_no_digit_cer"]
df = df.sort_values("our_no_digit_cer",ascending=False)
df.head()

,path,no_digit_label,our,google,our_no_digit_cer,google_no_digit_cer
524,/content/test-bahasa/wattpad-audio-wattpad-646.wav,who are you,ayu,how are you,0.666667,0.222222
189,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu pawesordantercetipawes,iaitu power order cathy paus,0.347826,0.521739
500,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ118-000005.wav,boleh fuckers quora sila tutup fuck tentang iq,boleh pakar sekolah sila tutup fak tentang itu,boleh fuckers kura sila tutup port tentang itu,0.307692,0.205128
267,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-ampe.wav,sebut perkataan ampe,sebuk pakatan ampa,sebut perkataan apa,0.277778,0.111111
37,/content/test-bahasa/-home-husein-speech-bahasa-tolong-sebut-jarum.wav,tolong sebut jarum,tolosebutjarol,tolong sebut jarum,0.250000,0.000000


our model not that good with english word because it not in the training set we use for community model

In [17]:
df = df.sort_values("google_no_digit_cer",ascending=False)
df.head()

,path,no_digit_label,our,google,our_no_digit_cer,google_no_digit_cer
28,/content/test-bahasa/wattpad-audio-wattpad-7.wav,bandung dua ribu tujuh belas,bandung dua ribu tujuh belas,bandung 2017,0.000000,0.708333
633,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1757.wav,dua puluh enam empat,dua puluh enam empat,26 empat,0.000000,0.705882
18,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-897.wav,phuket kochi dari rm tiga ratus sembilan puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,buket koci dari rm tiga ratus sembila puluh sembilan bali dari rm enam ratus dua puluh sembilan nanjing dari rm lima ratus lapan puluh sembilan,phuket kochi dari rm359 bali dari rm600 29 nanjing dari rm589,0.032787,0.672131
401,/content/test-bahasa/wattpad-audio-wattpad-544.wav,libras pov,liberas pov,libras theory,0.111111,0.555556
161,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-agihnya.wav,sebut perkataan agihnya,sebut perkataan agihnya,kata anaknya,0.000000,0.523810


most of the time google cer not good because it trying to predict digit for non digit dataset,that's why we provide separate dataset for fair comparison



# Digit dataset comparison

In [18]:
df = pd.DataFrame(zip(*[audios,with_digit_text,n_texts,g_texts,sk_d_cer,g_d_cer]))
df.columns = ["path","digit_label","our","google","our_digit_cer","google_digit_cer"]
df = df.sort_values("our_digit_cer",ascending=False)
df.head()

,path,digit_label,our,google,our_digit_cer,google_digit_cer
524,/content/test-bahasa/wattpad-audio-wattpad-646.wav,who are you,ayu,how are you,0.666667,0.222222
704,/content/test-bahasa/-home-husein-speech-bahasa-streaming-sementara empat ion bergabung.wav,sementara empat ion bergabung,sementara 4 ion penggabuh,sementara empat aeon bergabung,0.384615,0.076923
189,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu pawesordantercetipawes,iaitu power order cathy paus,0.347826,0.521739
89,/content/test-bahasa/-home-husein-speech-bahasa-streaming-sepanyol juta pada tahun.wav,sepanyol juta pada tahun,sepanyol 1000000 pada tahun,sepanyol juta pada tahun,0.333333,0.000000
500,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ118-000005.wav,boleh fuckers quora sila tutup fuck tentang iq,boleh pakar sekolah sila tutup fak tentang itu,boleh fuckers kura sila tutup port tentang itu,0.307692,0.205128


- it seems that in written form even though it is a number if not more than 9 should not convert to digit
- for juta(million) should not convert to digit too I guess(need to fix my normalization rule)

let see what kind of error for google

In [19]:
df = df.sort_values("google_digit_cer",ascending=False)
df.head()

,path,digit_label,our,google,our_digit_cer,google_digit_cer
633,/content/test-bahasa/-home-husein-speech-bahasa-news-audio-1757.wav,26 4,26 4,26 empat,0.000000,1.666667
401,/content/test-bahasa/wattpad-audio-wattpad-544.wav,libras pov,liberas pov,libras theory,0.111111,0.555556
161,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-agihnya.wav,sebut perkataan agihnya,sebut perkataan agihnya,kata anaknya,0.000000,0.523810
189,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu pawesordantercetipawes,iaitu power order cathy paus,0.347826,0.521739
10,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-ajal.wav,sebut perkataan ajal,sebut perkataan ajar,sebutkan saja,0.055556,0.444444


- no comment except that google wrongly predict the text

# Disclaimer

- This benchmark only on Google Speech To Text that is free, the paid version might be better than this
- The performance might varies if we test with different test set and domain(TODO)